In [2]:
# https://medium.com/datadriveninvestor/create-the-ultimate-stock-investing-portfolio-with-machine-learning-1f8034648211
# https://medium.com/datadriveninvestor/maximize-your-stock-market-earnings-with-machine-learning-87709494b1e3
# https://medium.com/datadriveninvestor/elons-lethal-mistake-predicting-the-stock-price-of-tesla-with-twitter-and-machine-learning-5e89282ce75f

In [1]:
import requests
import pandas as pd 
from yahoo_fin import stock_info as si 
from pandas_datareader import DataReader
import numpy as np

from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [55]:
ibovespa = pd.read_html("https://pt.wikipedia.org/wiki/Lista_de_companhias_citadas_no_Ibovespa")[0]
print(ibovespa.head())
ibovespa.columns = ["Symbol", "Share", "Sector", "Type", "Site"]
ibovespa = sorted(ibovespa.Symbol.tolist())

  Código             Ação                                              Setor  \
0  ABEV3        AMBEV S/A  Consumo não Cíclico / Bebidas / Cervejas e Ref...   
1  AZUL4             AZUL   Bens Industriais / Transporte / Transporte Aéreo   
2  B3SA3               B3  Financeiro e Outros / Serviços Financeiros Div...   
3  BBAS3  BANCO DO BRASIL  Financeiro e Outros / Intermediários Financeir...   
4  BBDC3         BRADESCO  Financeiro e Outros / Intermediários Financeir...   

        Tipo                         Site  
0         ON       http://ri.ambev.com.br  
1      PN N2  https://www.voeazul.com.br/  
2      ON NM        http://www.b3.com.br/  
3  ON ERJ NM        http://www.bb.com.br/  
4      ON N1  http://www.bradesco.com.br/  


In [56]:
tickers = ibovespa

In [57]:
from string import digits

#for i in range(0, len(tickers)):
#    tickers[i] = tickers[i].replace(' ', '.')
#    tickers[i] = tickers[i].replace('/', '')

#for i in range(0, len(tickers)):
#    tickers[i] = tickers[i]+".SA"

for i in range(0, len(tickers)):
    remove_digits = str.maketrans('', '', digits)
    tickers[i] = tickers[i].translate(remove_digits)

In [58]:
tickers

['ABEV',
 'AZUL',
 'BSA',
 'BBAS',
 'BBDC',
 'BBDC',
 'BBSE',
 'BEEF',
 'BPAC',
 'BRAP',
 'BRDT',
 'BRFS',
 'BRKM',
 'BRML',
 'BTOW',
 'CCRO',
 'CIEL',
 'CMIG',
 'COGN',
 'CPFE',
 'CRFB',
 'CSAN',
 'CSNA',
 'CVCB',
 'CYRE',
 'ECOR',
 'EGIE',
 'ELET',
 'ELET',
 'EMBR',
 'ENBR',
 'ENGI',
 'EQTL',
 'FLRY',
 'GGBR',
 'GNDI',
 'GOAU',
 'GOLL',
 'HAPV',
 'HGTX',
 'HYPE',
 'IGTA',
 'IRBR',
 'ITSA',
 'ITUB',
 'JBSS',
 'KLBN',
 'LAME',
 'LREN',
 'MGLU',
 'MRFG',
 'MRVE',
 'MULT',
 'NTCO',
 'PCAR',
 'PETR',
 'PETR',
 'QUAL',
 'RADL',
 'RAIL',
 'RENT',
 'SANB',
 'SBSP',
 'SULA',
 'SUZB',
 'TAEE',
 'TIMP',
 'TOTS',
 'UGPA',
 'USIM',
 'VALE',
 'VIVT',
 'VVAR',
 'WEGE',
 'YDUQ']

In [59]:
tickers[0]

'ABEV'

In [60]:
len(tickers)

75

In [62]:
#tickers = si.tickers_sp500()
recommendations = []

for ticker in tickers:
    lhs_url = 'https://query2.finance.yahoo.com/v10/finance/quoteSummary/'
    rhs_url = '?modules=upgradeDowngradeHistory,recommendationTrend,' \
              'financialData,earningsHistory,earningsTrend,industryTrend&' \
              'corsDomain=finance.yahoo.com'
              
    url =  lhs_url + ticker + rhs_url
    r = requests.get(url)
    if not r.ok:
        recommendation = 6
    try:
        result = r.json()['quoteSummary']['result'][0]
        recommendation =result['financialData']['recommendationMean']['fmt']
    except:
        recommendation = 6
    
    recommendations.append(recommendation)

In [2]:

tickers = si.tickers_sp500()
recommendations = []

for ticker in tickers:
    lhs_url = 'https://query2.finance.yahoo.com/v10/finance/quoteSummary/'
    rhs_url = '?formatted=true&crumb=swg7qs5y9UP&lang=en-US&region=US&' \
              'modules=upgradeDowngradeHistory,recommendationTrend,' \
              'financialData,earningsHistory,earningsTrend,industryTrend&' \
              'corsDomain=finance.yahoo.com'
              
    url =  lhs_url + ticker + rhs_url
    r = requests.get(url)
    if not r.ok:
        recommendation = 6
    try:
        result = r.json()['quoteSummary']['result'][0]
        recommendation =result['financialData']['recommendationMean']['fmt']
    except:
        recommendation = 6
    
    recommendations.append(recommendation)


In [15]:
df = pd.DataFrame()
df['Recommendations'] = recommendations
df['Company'] = tickers
df['Recommendations'] = pd.to_numeric(df['Recommendations'])
df = df[df.Recommendations != 6]
df.sort_values(by=['Recommendations'], ascending = True)

ValueError: Length of values does not match length of index

In [4]:
df.shape

(500, 2)

In [5]:
hold_df = df[df.Recommendations == 3]
buy_df = df[df.Recommendations <= 1.5]
sell_df = df[df.Recommendations >= 4.5]

df_list = [hold_df, buy_df, sell_df]
new_df = pd.concat(df_list)
new_df.reset_index(level=0, inplace=True)

In [6]:
# Get News
tickers =[] 
  
for index, rows in new_df.iterrows(): 
    tickers.append(rows.Company)

In [7]:
import nltk
nltk.download('vader_lexicon')

finwiz_url = 'https://finviz.com/quote.ashx?t='

news_tables = {}

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    response = urlopen(req)    
    html = BeautifulSoup(response)
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table
    
parsed_news = []

for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
    
        text = x.a.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]
        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text])
        
vader = SentimentIntensityAnalyzer()

columns = ['ticker', 'date', 'time', 'headline']

parsed_and_scored_news = pd.DataFrame(parsed_news, columns=columns)

parsed_and_scored_news['date'] = pd.to_datetime(parsed_and_scored_news.date).dt.date

parsed_and_scored_news.head()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/caiquecoelho/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,ticker,date,time,headline
0,CHRW,2020-10-04,09:41AM,"C.H. Robinson Worldwide, Inc.'s (NASDAQ:CHRW) ..."
1,CHRW,2020-09-28,04:15PM,C.H. Robinson Names Kermit Crawford to Board o...
2,CHRW,2020-09-28,12:34PM,"Should I Avoid C.H. Robinson Worldwide, Inc. (..."
3,CHRW,2020-09-27,01:30PM,Top 10 Logistics Companies in the World
4,CHRW,2020-09-24,10:36AM,"Benzinga's Top Upgrades, Downgrades For Septem..."


In [8]:
parsed_and_scored_news = parsed_and_scored_news.groupby(['ticker'], as_index = False).agg({'headline': ''.join}, Inplace=True)

In [9]:
parsed_and_scored_news.head()

,ticker,headline
0,BIO,3 Health Care Companies to Consider as Covid-1...
1,CHRW,"C.H. Robinson Worldwide, Inc.'s (NASDAQ:CHRW) ..."
2,CINF,Cincinnati Indemnity Company (The) -- Moody's ...
3,CMA,Comerica Bank's California Index Ticked UpCome...
4,CPB,Campbell (CPB) Up 0.6% Since Last Earnings Rep...


In [10]:
vader = SentimentIntensityAnalyzer()
scores = parsed_and_scored_news['headline'].apply(vader.polarity_scores).tolist()
scores_df = pd.DataFrame(scores)
parsed_and_scored_news = parsed_and_scored_news.join(scores_df, rsuffix='_right')

In [13]:
parsed_and_scored_news.head()

,ticker,headline,neg,neu,pos,compound
0,BIO,3 Health Care Companies to Consider as Covid-1...,0.024,0.874,0.103,0.9974
1,CHRW,"C.H. Robinson Worldwide, Inc.'s (NASDAQ:CHRW) ...",0.027,0.897,0.077,0.9920
2,CINF,Cincinnati Indemnity Company (The) -- Moody's ...,0.041,0.896,0.063,0.9681
3,CMA,Comerica Bank's California Index Ticked UpCome...,0.028,0.851,0.121,0.9976
4,CPB,Campbell (CPB) Up 0.6% Since Last Earnings Rep...,0.038,0.846,0.116,0.9971


In [14]:
parsed_and_scored_news

,ticker,headline,neg,neu,pos,compound
0,BIO,3 Health Care Companies to Consider as Covid-1...,0.024,0.874,0.103,0.9974
1,CHRW,"C.H. Robinson Worldwide, Inc.'s (NASDAQ:CHRW) ...",0.027,0.897,0.077,0.9920
2,CINF,Cincinnati Indemnity Company (The) -- Moody's ...,0.041,0.896,0.063,0.9681
3,CMA,Comerica Bank's California Index Ticked UpCome...,0.028,0.851,0.121,0.9976
4,CPB,Campbell (CPB) Up 0.6% Since Last Earnings Rep...,0.038,0.846,0.116,0.9971
5,DISCK,Discovery Selects Magnite to Strengthen DTC Ad...,0.031,0.892,0.077,0.9922
6,EQR,Equity Residential Announces Third Quarter 202...,0.014,0.882,0.104,0.9978
7,GPS,Heres How Gaps Analyst Day Can Boost the Stock...,0.042,0.863,0.096,0.9940
8,HRL,"Hormel Foods Joins Dine, Shop & Share for No K...",0.037,0.867,0.096,0.9949
9,INTC,"Here's How Much Investing $1,000 In The 5 Bigg...",0.066,0.798,0.137,0.9985


In [12]:
'''
from fbchat import log, Client

# Subclass fbchat.Client and override required methods
class EchoBot(Client):
    def onMessage(self, author_id, message_object, thread_id, thread_type, **kwargs):
        self.markAsDelivered(thread_id, message_object.uid)
        self.markAsRead(thread_id)

        log.info("{} from {} in {}".format(message_object, thread_id, thread_type.name))

        # If you're not the author, echo
        if author_id != self.uid:
            self.send("message", thread_id=thread_id, thread_type=thread_type)


client = EchoBot("<username>", "<Password>")
client.listen()
'''

'\nfrom fbchat import log, Client\n\n# Subclass fbchat.Client and override required methods\nclass EchoBot(Client):\n    def onMessage(self, author_id, message_object, thread_id, thread_type, **kwargs):\n        self.markAsDelivered(thread_id, message_object.uid)\n        self.markAsRead(thread_id)\n\n        log.info("{} from {} in {}".format(message_object, thread_id, thread_type.name))\n\n        # If you\'re not the author, echo\n        if author_id != self.uid:\n            self.send("message", thread_id=thread_id, thread_type=thread_type)\n\n\nclient = EchoBot("<username>", "<Password>")\nclient.listen()\n'